# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [7]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Characterizing Hydrodynamics and Hyporheic Imp...,oral presentation,phd-defence,SUNY-ESF PhD defense,1/7/2012,"Syracuse, NY",https://1drv.ms/b/s!Ao47KtQYIZUrsTyJIZRL1MSJPk...,NaN
1,Evaluation of the TRMM Real Time Multi-satelli...,poster,japan-trmm,TRMM conference,1/11/2012,Japan,https://1drv.ms/b/s!Ao47KtQYIZUrsTsc6MGwxNHdX_...,NaN
2,Macro Scale Hydrologic Modeling for Water Mana...,poster,AGU,2013 AGU,1/12/2013,"San Francisco, CA",https://1drv.ms/b/s!Ao47KtQYIZUrsTnqCvVYETev35...,NaN
3,A Global Drought Information System (GDIS) bas...,poster,WaterSympo,University of Washington Water symposium,1/9/2013,"Seattle, WA",https://1drv.ms/b/s!Ao47KtQYIZUrsTqmpE4f0FeIOB...,NaN
4,Reservoir in Global Water Cycle: Macro Scale H...,poster,AGU,2014 AGU,1/12/2014,"San Francisco, CA",https://1drv.ms/b/s!Ao47KtQYIZUrsTO4XzP_l59JFl...,NaN
5,Evaluation of Real-time Satellite Precipitatio...,oral presentation,workshop,UW CEE workshop,1/9/2014,"Seattle, WA",https://1drv.ms/b/s!Ao47KtQYIZUrsTKvX02ALoxXXV...,NaN
6,Quantifying Hyporheic Exchange Dynamics in a H...,poster,AGU,2016 AGU,1/12/2016,"San Francisco, CA",https://1drv.ms/b/s!Ao47KtQYIZUrsTb1vS8AsP487r...,NaN
7,The contribution of reservoirs to global land ...,oral presentation,CAS,chinese academy of sciences,1/2/2015,"Beijing, China",https://1drv.ms/b/s!Ao47KtQYIZUrsTUoHkMJazuygu...,NaN
8,Human Impacts on Hydrological Processes,oral presentation,PNNL,PNNL job interview,16/7/2015,"Richland, WA",https://1drv.ms/b/s!Ao47KtQYIZUrsTT5OBZs52uzLS...,NaN
9,"Reservoirs, sea level, and land waters",oral presentation,paper_reading,PNNL group seminar,26/1/2016,"Richland, WA",https://1drv.ms/b/s!Ao47KtQYIZUriz4qNQ6VU4u5kJ...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [8]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [9]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[See slides/poster](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2012-03-01-talk-1.md	  2014-02-01-talk-2.md
2013-03-01-tutorial-1.md  2014-03-01-talk-3.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
